In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\nguye\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# import torch
# import torch.nn.functional as F
import pandas as pd
import numpy as np
import tqdm
# from dgl import DGLGraph
import json
import pickle
import os
import spacy

## Data preparation

In [7]:
class ccf_ai(object):
    def __init__(self, dir):
        super(ccf_ai, self).__init__()
        self.dir = dir
        db_raw = open(self.dir + 'AI_collection.txt').readlines()
        nlp = spacy.load('en_core_web_lg')
        self.vec_paper = np.zeros((len(db_raw), 300))
        
        self.paper_w = np.zeros(len(db_raw))
        # ccf_set = np.zeros(len(db_raw), dtype=int)
        self.papers = []
        self.paper_ids_map = {}
        self.authors = []
        self.author_ids_map = {}
        ######### pd


        trange = tqdm.tqdm(range(len(db_raw)))
        for i in trange: # tqdm range
            sum_w = 0.
            d = json.loads(db_raw[i]) # one data sample

            paper_id = ''
            paper_title = ''
            paper_year = 0
            paper_venue = ''
            fos_pair = []

            if 'fos' in d:
                fos = d['fos']
                for f in fos:
                    fos_pair.append((f['name'], f['w'])) 
                    self.vec_paper[i, :] = self.vec_paper[i, :] + \
                                          nlp(f['name']).vector * f['w']
                    sum_w += f['w']
                self.vec_paper[i, :] = self.vec_paper[i, :] / sum_w
                self.paper_w[i] = sum_w # paper weight??

            if 'venue' in d:
                paper_venue = d['venue']['raw']

            if 'year' in d:
                paper_year = d['year']

            if 'id' in d:
                paper_id = d['id']
                if paper_id not in self.paper_ids_map:
                    self.paper_ids_map[paper_id] = len(self.papers) # map paper id to dataset id

            if 'title' in d:
                paper_title = d['title']

            self.papers.append({
                'gid': i,
                'id': paper_id,
                'title': paper_title,
                'year': paper_year,
                'venue': paper_venue,
                'fos': fos_pair
            })

            if 'authors' in d:
                da = d['authors']
                for j in da:
                    if j['id'] not in self.author_ids_map:
                        self.authors.append({
                            'gid': len(self.author_ids_map) + len(db_raw),
                            'id': j['id'],
                            'name': j['name']
                        })
                        self.author_ids_map[j['id']] = len(self.authors) - 1

        self.papers = pd.DataFrame(self.papers).set_index('gid').astype({'year': 'category','venue': 'category'})
        self.authors = pd.DataFrame(self.authors).set_index('gid').astype('category')

        self.edge_list = []
        self.edge_wt = []
        self.edge_ref = []
        self.author_write = [[] for i in range(len(self.author_ids_map))]
        self.author_ref = [[] for i in range(len(self.author_ids_map))]
        self.author_w_link = [[] for i in range(len(self.author_ids_map))] # set of all edge for an author in author_paper graph
        self.author_r_link = [[] for i in range(len(self.author_ids_map))] # set of all edge for an author in author_author graph

        self.paper_ref_paper = [[] for i in range(len(self.paper_ids_map))]  # idx of paper
        self.paper_author = [[] for i in range(len(self.paper_ids_map))]
        self.paper_link = [[] for i in range(len(self.paper_ids_map))]

        cnt = 0
        for i in tqdm.tqdm(range(len(db_raw))):
            d = json.loads(db_raw[i])
            ref = []
            if 'references' in d:
                ref = d['references']
            auth = []
            if 'authors' in d:
                auth = d['authors']
            for j in auth:
                if i not in self.author_write[self.author_ids_map[j['id']]]:
                    self.author_write[self.author_ids_map[j['id']]].append(i)

                self.paper_author[i].append(self.author_ids_map[j['id']])
                for k in ref:
                    if k in self.paper_ids_map:
                        if self.paper_ids_map[k] not in self.author_ref[self.author_ids_map[j['id']]]:
                            self.author_ref[self.author_ids_map[j['id']]].append(self.paper_ids_map[k])
                        self.paper_ref_paper[i].append(self.paper_ids_map[k])

        for i in range(len(self.author_ids_map)):
            for j in range(len(self.author_write[i])):
                tmp_write = self.author_write[i][j]
                self.author_w_link[i].append(cnt)
                self.paper_link[tmp_write].append(cnt)
                self.edge_wt.append((i + len(self.paper_ids_map), tmp_write, cnt))
                cnt += 1

        for i in range(len(self.author_ids_map)):
            for j in range(len(self.author_ref[i])):
                tmp_ref = self.author_ref[i][j]
                if tmp_ref not in self.author_write[i]:
                    self.author_r_link[i].append(cnt)
                    self.paper_link[tmp_ref].append(cnt)
                    self.edge_ref.append((i + len(self.paper_ids_map), tmp_ref, cnt))
                    cnt += 1

        print('Count: ', cnt)

        self.edge_list = self.edge_wt + self.edge_ref

        self.writes = pd.DataFrame(self.edge_wt, columns=['idx_A', 'idx_P', 'gid']).set_index('gid')
        self.writes['rt'] = True
        self.refs = pd.DataFrame(self.edge_ref, columns=['idx_A', 'idx_P', 'gid']).set_index('gid')
        self.refs['rt'] = False
        self.links = pd.concat([self.writes, self.refs], axis=0)
        


In [8]:
db = ccf_ai('D:/Documents/Webmining/P2R-GCN/data/')
db.papers

100%|██████████| 3590/3590 [00:00<00:00, 8628.51it/s]


Count:  106188


,id,title,year,venue,fos
gid,,,,,
0,1483870316,"You Only Look Once: Unified, Real-Time Object ...",2016,computer vision and pattern recognition,"[(Machine learning, 0.43618834), (Object-class..."
1,1537698211,Human Pose Estimation with Iterative Error Fee...,2016,computer vision and pattern recognition,"[(Computer vision, 0.4502649), (Machine learni..."
2,1552576688,Minimum error bounded efficient ℓ1 tracker wit...,2011,computer vision and pattern recognition,"[(Linear least squares, 0.476929843), (Mathema..."
3,1569892065,Efficient Piecewise Training of Deep Structure...,2016,computer vision and pattern recognition,"[(Image segmentation, 0.5859425), (Conditional..."
4,1573040851,Jointly Modeling Embedding and Translation to ...,2016,computer vision and pattern recognition,"[(Natural language, 0.496494561), (Semantics, ..."
...,...,...,...,...,...
3585,764651262,FlowNet: Learning Optical Flow with Convolutio...,2015,international conference on computer vision,"[(Supervised learning, 0.5532485), (Pattern re..."
3586,845365781,Holistically-Nested Edge Detection,2015,international conference on computer vision,"[(Feature (computer vision), 0.6397062), (Comp..."
3587,846669277,Differential Recurrent Neural Networks for Act...,2015,international conference on computer vision,"[(Computer vision, 0.4164644), (Machine learni..."


In [9]:
db.authors

,id,name
gid,,
3590,2392241600,Joseph Redmon
3591,2310010008,Santosh Kumar Divvala
3592,2473549963,Ross B. Girshick
3593,1988090614,Ali Farhadi
3594,2429081565,Joao Carreira
...,...,...
9058,771585220,Caner Hazirbas
9059,337396571,V. Golkov
9060,1163702811,Patrick van der Smagt


In [10]:
db.refs

,idx_A,idx_P,rt
gid,,,
12281,3590,2396,False
12282,3590,2982,False
12283,3590,2992,False
12284,3590,2410,False
12285,3590,3014,False
...,...,...,...
106183,9062,933,False
106184,9062,979,False
106185,9062,988,False


In [11]:
db.writes

,idx_A,idx_P,rt
gid,,,
0,3590,0,True
1,3590,1448,True
2,3591,0,True
3,3591,26,True
4,3591,488,True
...,...,...,...
12276,9058,3585,True
12277,9059,3585,True
12278,9060,3585,True


In [12]:
df_interaction = db.links
df_interaction["inter_score"] = 1
df_interaction.head()

,idx_A,idx_P,rt,inter_score
gid,,,,
0,3590,0,True,1
1,3590,1448,True,1
2,3591,0,True,1
3,3591,26,True,1
4,3591,488,True,1


In [13]:
df_all = df_interaction.copy()
df_all[['idx_A', 'idx_P']] = df_all[['idx_A', 'idx_P']].astype(str)
df_all

,idx_A,idx_P,rt,inter_score
gid,,,,
0,3590,0,True,1
1,3590,1448,True,1
2,3591,0,True,1
3,3591,26,True,1
4,3591,488,True,1
...,...,...,...,...
106183,9062,933,False,1
106184,9062,979,False,1
106185,9062,988,False,1


## Define metrics

In [59]:
# Metric
import numpy as np
import math

def getHitRatio(ranklist, gtItem):
    if gtItem in ranklist:
        return 1
    return 0

def get_rr(ranklist, gtItem):
    if gtItem in ranklist:
        rank = np.where(ranklist == gtItem)[0][0] + 1
        return 1/(rank)
    else:
        return 1/len(ranklist)
def getNDCG(ranklist, gtItem):
    ar = np.array(ranklist)
    if gtItem in ar:
        return math.log(2) / math.log(np.where(ar == gtItem)[0][0] + 2)
    return 0

## Data partition

In [15]:
# training testing split
# def train_test_split(df):
retrieve_name = df_all['idx_A'].value_counts()[df_all['idx_A'].value_counts() > 10].keys()
df_test = df_all[df_all['idx_A'].isin(retrieve_name)]
df_test = df_test.groupby('idx_A').tail(1)
df_train = df_all.drop(index=df_test.index)
df_train.shape, df_test.shape

((103286, 4), (2902, 4))

In [16]:
len(set(df_all["idx_A"].unique()).difference(sorted(df_all.groupby("idx_A")['idx_P'].apply(list)[3590])))


5473

In [17]:
import random

random.seed(73)

def get_sampling(df_target, df_source, numbers_of_N_sample, generate_negative = False):
    '''
    1.
    return: test_rating in the shape of [ [user_pid,article] * N ]
    2.
    from the target dataframe, sampling the negative sample from source dataframe
    return: list in the shape of [ [((user_id, ground truth)) , [numbers_of_N_sample(negative samples)]] * N]
    '''
    test_rating = df_target[['idx_A','idx_P']].values.tolist()
    print('test_rating yield successfully!!!')
    
    if generate_negative:
        act_set = set(df_source["idx_P"].unique())
        mapping_user_act = df_source.groupby("idx_A")['idx_P'].apply(list)
        negative_test_rating = []
        for i in range(df_target.shape[0]):
            drop_id = test_rating[i][0]
            list_ = random.sample(list(act_set.difference(mapping_user_act[drop_id])), numbers_of_N_sample)
            negative_test_rating.append([test_rating[i], list_])
        print('negative_test_rating yield successfully!!!')

        return test_rating, negative_test_rating
    else:
        return test_rating

In [18]:
test_rating, negative_test_rating = get_sampling(df_test, df_train, 19, True)
# print(negative_test_rating)


test_rating yield successfully!!!
negative_test_rating yield successfully!!!


In [19]:
user_id_map = {i:v for i, v in enumerate(df_all["idx_A"].unique())}
inverse_user_id_map = {v:i for i, v in enumerate(df_all["idx_A"].unique())}
activity_id_map = {i:v for i, v in enumerate(df_all["idx_P"].unique())}
inverse_activity_id_map = {v:i for i, v in enumerate(df_all["idx_P"].unique())}
# print(user_id_map)
# print(inverse_activity_id_map)

In [20]:
df_train_map = df_train.copy()
df_test_map = df_test.copy()

df_train_map['idx_A'] = df_train_map['idx_A'].map(inverse_user_id_map)
df_train_map['idx_P'] = df_train_map['idx_P'].map(inverse_activity_id_map)
df_test_map['idx_A'] = df_test_map['idx_A'].map(inverse_user_id_map)
df_test_map['idx_P'] = df_test_map['idx_P'].map(inverse_activity_id_map)

### creating a mapping list for testing data   
test_rating_map = []
for i in range(len(test_rating)):
    test_rating_map.append([inverse_user_id_map[test_rating[i][0]],inverse_activity_id_map[test_rating[i][1]]])


### creating a mapping list for negative testing data
negative_test_rating_map = []
for i in range(len(negative_test_rating)):
    negative_test_rating_map.append([
                                     [inverse_user_id_map[negative_test_rating[i][0][0]],
                                      inverse_activity_id_map[negative_test_rating[i][0][1]]],
                                     list(map(lambda w: inverse_activity_id_map[w], negative_test_rating[i][1]))])

In [21]:
print('df_train_group_map shape: ',df_train_map.shape)
print('number of users: ', len(user_id_map))
print('number of items: ', len(activity_id_map))
print('The sparse matrix is one with shape (%d , %d), with %d non-zero read_times'
      %(len(user_id_map),len(activity_id_map), df_train_map.shape[0]))

df_train_group_map shape:  (103286, 4)
number of users:  5473
number of items:  3590
The sparse matrix is one with shape (5473 , 3590), with 103286 non-zero read_times


In [22]:
nb_users = len(user_id_map)
nb_articles = len(activity_id_map)
uim = np.zeros((nb_users, nb_articles), dtype=np.float32)
uim[df_train_map["idx_A"], df_train_map["idx_P"]] = df_train_map["inter_score"]

## Alternating Least Squares

In [65]:
from scipy import sparse as sp
from implicit.als import AlternatingLeastSquares

uim = sp.csr_matrix(uim)
print('uim shape :', uim.shape)

model = AlternatingLeastSquares(factors=16, 
                                random_state=42,
                                num_threads=16,
                                alpha=128)

model.fit(user_items=uim, show_progress=True)

uim shape : (5473, 3590)


100%|██████████| 15/15 [00:01<00:00,  9.19it/s]


In [66]:
model.user_factors.shape[0], model.item_factors.shape[0]


(5473, 3590)

In [67]:
from tqdm import tqdm

topK = 5
hits = []
ndcgs = []
rrs = []
for idx in tqdm(range(len(test_rating_map))):
    rating = test_rating_map[idx]
    items = negative_test_rating_map[idx][1]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    predictions = model.rank_items(u, uim, items)
    ranklist = predictions[0][:topK]
    rr = get_rr(predictions[0], gtItem)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    hits.append(hr)
    ndcgs.append(ndcg)
    rrs.append(rr)

  0%|          | 0/2902 [00:00<?, ?it/s]C:\Users\nguye\AppData\Local\Temp\ipykernel_15236\482312308.py:13: DeprecationWarning: rank_items is deprecated. Use recommend with the 'items' parameter instead
  predictions = model.rank_items(u, uim, items)
100%|██████████| 2902/2902 [00:00<00:00, 8172.03it/s]


In [68]:
print(np.array(hits).mean())
print(np.array(rrs).mean())

0.937973811164714
0.7777636933477733


## LogisticMatrixFactorization

In [81]:
from scipy import sparse as sp
from implicit.lmf import LogisticMatrixFactorization

uim = sp.csr_matrix(uim)
print('uim shape :', uim.shape)

model = LogisticMatrixFactorization(factors=256,
                                    learning_rate=0.5,
                                    iterations=200,
                                    num_threads=16,
                                    random_state=42)

model.fit(user_items=uim, show_progress=True)

uim shape : (5473, 3590)


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [02:18<00:00,  1.45it/s]


In [82]:
from tqdm import tqdm

topK = 5
hits = []
ndcgs = []
rrs = []

for idx in tqdm(range(len(test_rating_map))):
    rating = test_rating_map[idx]
    items = negative_test_rating_map[idx][1]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    predictions = model.rank_items(u, uim, items)
    ranklist = predictions[0][:topK]
    rr = get_rr(predictions[0], gtItem)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    hits.append(hr)
    ndcgs.append(ndcg)
    rrs.append(rr)

  0%|          | 0/2902 [00:00<?, ?it/s]C:\Users\nguye\AppData\Local\Temp\ipykernel_15236\1212328913.py:14: DeprecationWarning: rank_items is deprecated. Use recommend with the 'items' parameter instead
  predictions = model.rank_items(u, uim, items)
100%|██████████| 2902/2902 [00:00<00:00, 7660.72it/s]


In [83]:
print(np.array(hits).mean())
print(np.array(rrs).mean())

0.8108201240523777
0.6107792381827529
